# Solver Testing Notebook

This notebook tests the coffee inventory optimization solver with multiple scenarios and visualizes the results.

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML

from solver import SolverInput, SolverOutput, solve, SolverFail

In [2]:
def solution_to_dataframe(sol: SolverOutput, inp: SolverInput) -> pd.DataFrame:
    days = list(range(1, inp.T + 1))
    
    daily_purchase_cost = [inp.P[t] * sol.x[t] for t in range(inp.T)]
    daily_transport_cost = [inp.C if sol.y[t] == 1 else 0 for t in range(inp.T)]
    daily_total_cost = [daily_purchase_cost[t] + daily_transport_cost[t] for t in range(inp.T)]
    
    return pd.DataFrame({
        'Day': days,
        'Demand (kg)': inp.D,
        'Ordered (kg)': sol.x,
        'Order Placed': sol.y,
        'Inventory (kg)': sol.I,
        'Purchase Cost (PLN)': daily_purchase_cost,
        'Transport Cost (PLN)': daily_transport_cost,
        'Total Daily Cost (PLN)': daily_total_cost,
    })


def plot_scenario(scenario_name: str, sol: SolverOutput, inp: SolverInput, df: pd.DataFrame):
    days = list(range(1, inp.T + 1))
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Demand vs. Inventory",
            "Order Amounts",
            "Daily Costs",
            "Cumulative Cost"
        ),
        specs=[[{"secondary_y": True}, {}],
               [{}, {}]]
    )
    
    # Panel 1: Demand vs Inventory
    fig.add_trace(
        go.Bar(x=days, y=inp.D, name="Demand", marker_color='lightblue', opacity=0.7),
        row=1, col=1, secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=days, y=sol.I, name="Inventory", mode='lines+markers', 
                   line=dict(color='darkblue', width=2)),
        row=1, col=1, secondary_y=True
    )
    fig.update_xaxes(title_text="Day", row=1, col=1)
    fig.update_yaxes(title_text="Demand (kg)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Inventory (kg)", row=1, col=1, secondary_y=True)
    
    # Panel 2: Order Amounts
    fig.add_trace(
        go.Bar(x=days, y=sol.x, name="Order Amount", marker_color='green', opacity=0.7),
        row=1, col=2
    )
    fig.update_xaxes(title_text="Day", row=1, col=2)
    fig.update_yaxes(title_text="Order Amount (kg)", row=1, col=2)

    
    # Panel 3: Daily Costs
    fig.add_trace(
        go.Bar(x=days, y=df['Purchase Cost (PLN)'], name="Purchase Cost", 
               marker_color='orange', opacity=0.7),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=days, y=df['Transport Cost (PLN)'], name="Transport Cost", 
               marker_color='red', opacity=0.7),
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Day", row=2, col=1)
    fig.update_yaxes(title_text="Cost (PLN)", row=2, col=1)
    
    # Panel 4: Cumulative Cost
    cumulative_cost = df['Total Daily Cost (PLN)'].cumsum()
    fig.add_trace(
        go.Scatter(x=days, y=cumulative_cost, name="Cumulative Cost", 
                   mode='lines+markers', line=dict(color='purple', width=2),
                   fill='tozeroy'),
        row=2, col=2
    )
    fig.update_xaxes(title_text="Day", row=2, col=2)
    fig.update_yaxes(title_text="Cumulative Cost (PLN)", row=2, col=2)
    
    fig.update_layout(
        title_text=f"{scenario_name} - Solver Results (Objective: {sol.objective_value:.2f} PLN)",
        height=900,
        showlegend=True,
        barmode='stack'
    )
    
    fig.show()

## Test Scenario 1: Uniform Demand

Constant demand over 7 days with moderate pricing.

In [3]:
scenario_1 = {
    "V_max": 100.0,  # kg
    "P": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],  # PLN/kg
    "C": 50.0,  # PLN (transportation cost)
    "D": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],  # kg demand per day
    "I0": 30.0,  # kg initial stock
    "alpha": 0.1,  # 10% daily loss
    "T": 7
}

inp1 = SolverInput(**scenario_1)

try:
    sol1 = solve(inp1)
    df1 = solution_to_dataframe(sol1, inp1)
    print(f"Objective value: {sol1.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol1.y) if y == 1]}")
    display(HTML(df1.to_html(index=False)))
    plot_scenario("Scenario 1: Uniform Demand", sol1, inp1, df1)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol1 = None
    df1 = None

Objective value: 597.98 PLN
Orders placed on days: [3, 6]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,10.0,0.000000,0,17.000000,0.000000,0.0,0.000000
2,10.0,0.000000,0,5.300000,0.000000,0.0,0.000000
3,10.0,28.686790,1,23.456790,286.867901,50.0,336.867901
4,10.0,0.000000,0,11.111111,0.000000,0.0,0.000000
5,10.0,0.000000,0,0.000000,0.000000,0.0,0.000000
6,10.0,21.111111,1,11.111111,211.111111,50.0,261.111111
7,10.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 2: Peak Demand with Variable Pricing

High demand early in the week, lower prices on certain days.

In [4]:
# Scenario 2: Peak demand with variable pricing
scenario_2 = {
    "V_max": 150.0,  # kg
    "P": [12.0, 10.0, 14.0, 10.0, 13.0, 11.0, 15.0],  # Variable pricing (PLN/kg)
    "C": 50.0,  # PLN
    "D": [20.0, 25.0, 15.0, 10.0, 12.0, 18.0, 22.0],  # Variable demand
    "I0": 40.0,  # kg initial stock
    "alpha": 0.1,
    "T": 7
}

inp2 = SolverInput(**scenario_2)

try:
    sol2 = solve(inp2)
    df2 = solution_to_dataframe(sol2, inp2)
    print(f"Objective value: {sol2.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol2.y) if y == 1]}")
    display(HTML(df2.to_html(index=False)))
    plot_scenario("Scenario 2: Peak Demand", sol2, inp2, df2)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol2 = None
    df2 = None

Objective value: 1122.89 PLN
Orders placed on days: [2, 4, 6]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,20.0,0.000000,0,16.000000,0.000000,0.0,0.000000
2,25.0,27.266667,1,16.666667,272.666667,50.0,322.666667
3,15.0,0.000000,0,0.000000,0.000000,0.0,0.000000
4,10.0,23.333333,1,13.333333,233.333333,50.0,283.333333
5,12.0,0.000000,0,0.000000,0.000000,0.0,0.000000
6,18.0,42.444444,1,24.444444,466.888889,50.0,516.888889
7,22.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 3: High Loss Rate & Tight Capacity

Perishable product with high daily loss and limited warehouse capacity.

In [5]:
# Scenario 3: High loss rate with tight capacity
scenario_3 = {
    "V_max": 50.0,  # kg (tight)
    "P": [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0],  # Constant price
    "C": 80.0,  # High transportation cost
    "D": [12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0],  # Moderate constant demand
    "I0": 20.0,  # kg initial stock
    "alpha": 0.25,  # 25% daily loss (high perishability)
    "T": 7
}

inp3 = SolverInput(**scenario_3)

try:
    sol3 = solve(inp3)
    df3 = solution_to_dataframe(sol3, inp3)
    print(f"Objective value: {sol3.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol3.y) if y == 1]}")
    display(HTML(df3.to_html(index=False)))
    plot_scenario("Scenario 3: High Loss & Tight Capacity", sol3, inp3, df3)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol3 = None
    df3 = None

Objective value: 1466.25 PLN
Orders placed on days: [2, 4, 6]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,12.0,0.00,0,3.0,0.00,0.0,0.00
2,12.0,25.75,1,16.0,386.25,80.0,466.25
3,12.0,0.00,0,0.0,0.00,0.0,0.00
4,12.0,28.00,1,16.0,420.00,80.0,500.00
5,12.0,0.00,0,0.0,0.00,0.0,0.00
6,12.0,28.00,1,16.0,420.00,80.0,500.00
7,12.0,0.00,0,0.0,0.00,0.0,0.00


## Test Scenario 4: Demand Spike

Demand spikes on days 3 and 5 with lower pricing early in the week.

In [6]:
# Scenario 4: Weekend demand spike
scenario_4 = {
    "V_max": 120.0,  # kg
    "P": [8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0],  # Increasing prices through the week
    "C": 45.0,  # PLN
    "D": [8.0, 10.0, 30.0, 12.0, 35.0, 14.0, 18.0],  # Demand spike on specific days
    "I0": 25.0,  # kg initial stock
    "alpha": 0.1,
    "T": 7
}

inp4 = SolverInput(**scenario_4)

try:
    sol4 = solve(inp4)
    df4 = solution_to_dataframe(sol4, inp4)
    print(f"Objective value: {sol4.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol4.y) if y == 1]}")
    display(HTML(df4.to_html(index=False)))
    plot_scenario("Scenario 4: Weekend Demand Spike", sol4, inp4, df4)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol4 = None
    df4 = None

Objective value: 1349.37 PLN
Orders placed on days: [3]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,8.0,0.000000,0,14.500000,0.000000,0.0,0.000000
2,10.0,0.000000,0,3.050000,0.000000,0.0,0.000000
3,30.0,130.437442,1,103.182442,1304.374417,45.0,1349.374417
4,12.0,0.000000,0,80.864198,0.000000,0.0,0.000000
5,35.0,0.000000,0,37.777778,0.000000,0.0,0.000000
6,14.0,0.000000,0,20.000000,0.000000,0.0,0.000000
7,18.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 5: Price Arbitrage Opportunity

Day 3 has significantly lower price - tests if solver will buy ahead when prices drop.

In [7]:
# Scenario 5: Price arbitrage opportunity
scenario_5 = {
    "V_max": 200.0,  # Large capacity for stockpiling
    "P": [15.0, 14.0, 5.0, 16.0, 15.0, 14.0, 15.0],  # Day 3 has very low price
    "C": 60.0,  # PLN
    "D": [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0],  # Constant demand
    "I0": 10.0,  # Low initial stock
    "alpha": 0.05,  # Low loss to make arbitrage viable
    "T": 7
}

inp5 = SolverInput(**scenario_5)

try:
    sol5 = solve(inp5)
    df5 = solution_to_dataframe(sol5, inp5)
    print(f"Objective value: {sol5.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol5.y) if y == 1]}")
    display(HTML(df5.to_html(index=False)))
    plot_scenario("Scenario 5: Price Arbitrage", sol5, inp5, df5)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol5 = None
    df5 = None

Objective value: 855.95 PLN
Orders placed on days: [1, 3]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,15.0,21.289474,1,15.789474,319.342105,60.0,379.342105
2,15.0,0.000000,0,0.000000,0.000000,0.0,0.000000
3,15.0,83.321299,1,68.321299,416.606495,60.0,476.606495
4,15.0,0.000000,0,49.905234,0.000000,0.0,0.000000
5,15.0,0.000000,0,32.409972,0.000000,0.0,0.000000
6,15.0,0.000000,0,15.789474,0.000000,0.0,0.000000
7,15.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 6: High Transportation Cost

Very expensive transportation - solver should minimize number of orders.

In [8]:
# Scenario 6: High transportation cost
scenario_6 = {
    "V_max": 150.0,  # kg
    "P": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],  # Constant price
    "C": 200.0,  # Very high transportation cost
    "D": [12.0, 14.0, 11.0, 13.0, 15.0, 12.0, 10.0],  # Variable demand
    "I0": 15.0,  # kg initial stock
    "alpha": 0.08,  # Low loss
    "T": 7
}

inp6 = SolverInput(**scenario_6)

try:
    sol6 = solve(inp6)
    df6 = solution_to_dataframe(sol6, inp6)
    print(f"Objective value: {sol6.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol6.y) if y == 1]}")
    display(HTML(df6.to_html(index=False)))
    plot_scenario("Scenario 6: High Transportation Cost", sol6, inp6, df6)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol6 = None
    df6 = None

Objective value: 1108.46 PLN
Orders placed on days: [2]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,12.0,0.000000,0,1.800000,0.000000,0.0,0.000000
2,14.0,90.846074,1,78.502074,908.460741,200.0,1108.460741
3,11.0,0.000000,0,61.221908,0.000000,0.0,0.000000
4,13.0,0.000000,0,43.324156,0.000000,0.0,0.000000
5,15.0,0.000000,0,24.858223,0.000000,0.0,0.000000
6,12.0,0.000000,0,10.869565,0.000000,0.0,0.000000
7,10.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 7: Long-Term Planning (3 Weeks)

Planning for 21 days (3 weeks) starting from zero inventory with weekly demand patterns.

In [9]:
# Scenario 7: Long-term planning (3 weeks = 21 days)
# Weekly pattern: moderate Mon-Thu, high Fri-Sat, low Sun
weekly_demand_pattern = [12.0, 13.0, 14.0, 15.0, 20.0, 22.0, 8.0]  # One week
demand_21_days = weekly_demand_pattern * 3  # Repeat for 3 weeks

# Price fluctuations with weekly cycle (lower mid-week, higher weekends)
weekly_price_pattern = [11.0, 10.5, 10.0, 10.5, 12.0, 13.0, 11.5]
prices_21_days = weekly_price_pattern * 3

scenario_7 = {
    "V_max": 180.0,  # kg - needs larger capacity for longer planning
    "P": prices_21_days,  # 21-day price list
    "C": 70.0,  # PLN
    "D": demand_21_days,  # 21-day demand list
    "I0": 0.0,  # Start from ZERO inventory
    "alpha": 0.1,  # 10% daily loss
    "T": 21
}

inp7 = SolverInput(**scenario_7)

try:
    sol7 = solve(inp7)
    df7 = solution_to_dataframe(sol7, inp7)
    print(f"Objective value: {sol7.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol7.y) if y == 1]}")
    display(HTML(df7.to_html(index=False)))
    plot_scenario("Scenario 7: Long-Term Planning (3 Weeks)", sol7, inp7, df7)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol7 = None
    df7 = None

Objective value: 4224.56 PLN
Orders placed on days: [1, 3, 8, 10, 15, 17]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,12.0,26.444444,1,14.444444,290.888889,70.0,360.888889
2,13.0,0.000000,0,0.000000,0.000000,0.0,0.000000
3,14.0,97.729614,1,83.729614,977.296144,70.0,1047.296144
4,15.0,0.000000,0,60.356653,0.000000,0.0,0.000000
5,20.0,0.000000,0,34.320988,0.000000,0.0,0.000000
6,22.0,0.000000,0,8.888889,0.000000,0.0,0.000000
7,8.0,0.000000,0,0.000000,0.000000,0.0,0.000000
8,12.0,26.444444,1,14.444444,290.888889,70.0,360.888889
9,13.0,0.000000,0,0.000000,0.000000,0.0,0.000000
10,14.0,97.729614,1,83.729614,977.296144,70.0,1047.296144


## Test Scenario 8: Extreme Perishability (50% Daily Loss)

Highly perishable product with 50% daily loss - tests feasibility limits.

In [10]:
# Scenario 8: Extreme perishability
scenario_8 = {
    "V_max": 100.0,
    "P": [12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0],
    "C": 40.0,  # Lower transport cost to make frequent orders viable
    "D": [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0],
    "I0": 5.0,
    "alpha": 0.5,  # 50% daily loss!
    "T": 7
}

inp8 = SolverInput(**scenario_8)

try:
    sol8 = solve(inp8)
    df8 = solution_to_dataframe(sol8, inp8)
    print(f"Objective value: {sol8.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol8.y) if y == 1]}")
    display(HTML(df8.to_html(index=False)))
    plot_scenario("Scenario 8: Extreme Perishability", sol8, inp8, df8)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol8 = None
    df8 = None

Objective value: 1510.00 PLN
Orders placed on days: [1, 2, 3, 4, 5, 6, 7]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,15.0,12.5,1,0.0,150.0,40.0,190.0
2,15.0,15.0,1,0.0,180.0,40.0,220.0
3,15.0,15.0,1,0.0,180.0,40.0,220.0
4,15.0,15.0,1,0.0,180.0,40.0,220.0
5,15.0,15.0,1,0.0,180.0,40.0,220.0
6,15.0,15.0,1,0.0,180.0,40.0,220.0
7,15.0,15.0,1,0.0,180.0,40.0,220.0


## Test Scenario 9: Capacity Stress Test (Near-Zero Capacity)

Very tight warehouse capacity forces frequent ordering.

In [11]:
# Scenario 9: Capacity stress test
scenario_9 = {
    "V_max": 20.0,  # Very tight capacity
    "P": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
    "C": 35.0,
    "D": [15.0, 16.0, 14.0, 18.0, 17.0, 15.0, 16.0],  # High demand relative to capacity
    "I0": 10.0,
    "alpha": 0.1,
    "T": 7
}

inp9 = SolverInput(**scenario_9)

try:
    sol9 = solve(inp9)
    df9 = solution_to_dataframe(sol9, inp9)
    print(f"Objective value: {sol9.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol9.y) if y == 1]}")
    display(HTML(df9.to_html(index=False)))
    plot_scenario("Scenario 9: Capacity Stress Test", sol9, inp9, df9)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol9 = None
    df9 = None

Objective value: 1212.22 PLN
Orders placed on days: [1, 2, 4, 6]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,15.0,6.000000,1,0.000000,60.000000,35.0,95.000000
2,16.0,31.555556,1,15.555556,315.555556,35.0,350.555556
3,14.0,0.000000,0,0.000000,0.000000,0.0,0.000000
4,18.0,36.888889,1,18.888889,368.888889,35.0,403.888889
5,17.0,0.000000,0,0.000000,0.000000,0.0,0.000000
6,15.0,32.777778,1,17.777778,327.777778,35.0,362.777778
7,16.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 10: Price Volatility (Chaotic Pricing)

Wild price fluctuations with multiple arbitrage opportunities.

In [12]:
# Scenario 10: Price volatility
scenario_10 = {
    "V_max": 150.0,
    "P": [20.0, 8.0, 18.0, 6.0, 22.0, 9.0, 15.0],  # Chaotic pricing
    "C": 50.0,
    "D": [12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0],  # Constant demand for clarity
    "I0": 15.0,
    "alpha": 0.08,
    "T": 7
}

inp10 = SolverInput(**scenario_10)

try:
    sol10 = solve(inp10)
    df10 = solution_to_dataframe(sol10, inp10)
    print(f"Objective value: {sol10.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol10.y) if y == 1]}")
    display(HTML(df10.to_html(index=False)))
    plot_scenario("Scenario 10: Price Volatility", sol10, inp10, df10)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol10 = None
    df10 = None

Objective value: 614.89 PLN
Orders placed on days: [2, 4]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,12.0,0.000000,0,1.800000,0.000000,0.0,0.000000
2,12.0,23.387478,1,13.043478,187.099826,50.0,237.099826
3,12.0,0.000000,0,0.000000,0.000000,0.0,0.000000
4,12.0,54.631709,1,42.631709,327.790252,50.0,377.790252
5,12.0,0.000000,0,27.221172,0.000000,0.0,0.000000
6,12.0,0.000000,0,13.043478,0.000000,0.0,0.000000
7,12.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 11: Declining Demand Trend

End-of-season scenario with demand decreasing over time.

In [13]:
# Scenario 11: Declining demand trend
demand_declining = [25.0, 22.0, 19.0, 16.0, 12.0, 8.0, 5.0]  # Linear decline

scenario_11 = {
    "V_max": 120.0,
    "P": [11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0],
    "C": 55.0,
    "D": demand_declining,
    "I0": 30.0,  # Starting with some inventory
    "alpha": 0.1,
    "T": 7
}

inp11 = SolverInput(**scenario_11)

try:
    sol11 = solve(inp11)
    df11 = solution_to_dataframe(sol11, inp11)
    print(f"Objective value: {sol11.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol11.y) if y == 1]}")
    display(HTML(df11.to_html(index=False)))
    plot_scenario("Scenario 11: Declining Demand", sol11, inp11, df11)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol11 = None
    df11 = None

Objective value: 1071.18 PLN
Orders placed on days: [2, 4]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,25.0,0.000000e+00,0,2.000000e+00,0.000000e+00,0.0,0.000000e+00
2,22.0,4.131111e+01,1,2.111111e+01,4.544222e+02,55.0,5.094222e+02
3,19.0,2.167155e-13,0,1.492140e-13,2.383871e-12,0.0,2.383871e-12
4,16.0,4.606859e+01,1,3.006859e+01,5.067545e+02,55.0,5.617545e+02
5,12.0,0.000000e+00,0,1.506173e+01,0.000000e+00,0.0,0.000000e+00
6,8.0,0.000000e+00,0,5.555556e+00,0.000000e+00,0.0,0.000000e+00
7,5.0,0.000000e+00,0,0.000000e+00,0.000000e+00,0.0,0.000000e+00


## Test Scenario 12: Ramp-Up / Product Launch

New product introduction with growing demand over time.

In [14]:
# Scenario 12: Ramp-up / product launch
demand_ramping = [5.0, 8.0, 12.0, 16.0, 20.0, 23.0, 25.0]  # Growing demand

scenario_12 = {
    "V_max": 130.0,
    "P": [13.0, 12.5, 12.0, 11.5, 11.0, 10.5, 10.0],  # Decreasing prices (economies of scale)
    "C": 60.0,
    "D": demand_ramping,
    "I0": 20.0,  # Start with moderate inventory
    "alpha": 0.1,
    "T": 7
}

inp12 = SolverInput(**scenario_12)

try:
    sol12 = solve(inp12)
    df12 = solution_to_dataframe(sol12, inp12)
    print(f"Objective value: {sol12.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol12.y) if y == 1]}")
    display(HTML(df12.to_html(index=False)))
    plot_scenario("Scenario 12: Ramp-Up (Product Launch)", sol12, inp12, df12)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol12 = None
    df12 = None

Objective value: 1248.48 PLN
Orders placed on days: [3, 5, 7]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,5.0,0.000000,0,1.300000e+01,0.000000,0.0,0.000000
2,8.0,0.000000,0,3.700000e+00,0.000000,0.0,0.000000
3,12.0,26.447778,1,1.777778e+01,317.373333,60.0,377.373333
4,16.0,0.000000,0,0.000000e+00,0.000000,0.0,0.000000
5,20.0,45.555556,1,2.555556e+01,501.111111,60.0,561.111111
6,23.0,0.000000,0,-3.907985e-14,0.000000,0.0,0.000000
7,25.0,25.000000,1,0.000000e+00,250.000000,60.0,310.000000


## Test Scenario 13: Binary Demand (Conference/Event Days)

Demand only on specific days (conferences/events), zero on others.

In [15]:
# Scenario 13: Binary demand (events only on certain days)
scenario_13 = {
    "V_max": 100.0,
    "P": [11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0],
    "C": 50.0,
    "D": [0.0, 0.0, 35.0, 0.0, 40.0, 0.0, 30.0],  # Only days 3, 5, 7 have demand
    "I0": 10.0,
    "alpha": 0.15,  # Higher loss since inventory sits unused on zero-demand days
    "T": 7
}

inp13 = SolverInput(**scenario_13)

try:
    sol13 = solve(inp13)
    df13 = solution_to_dataframe(sol13, inp13)
    print(f"Objective value: {sol13.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol13.y) if y == 1]}")
    display(HTML(df13.to_html(index=False)))
    plot_scenario("Scenario 13: Binary Demand (Events)", sol13, inp13, df13)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol13 = None
    df13 = None

Objective value: 1237.45 PLN
Orders placed on days: [3, 5, 7]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,0.0,0.00000,0,8.500,0.00000,0.0,0.00000
2,0.0,0.00000,0,7.225,0.00000,0.0,0.00000
3,35.0,28.85875,1,0.000,317.44625,50.0,367.44625
4,0.0,0.00000,0,0.000,0.00000,0.0,0.00000
5,40.0,40.00000,1,0.000,440.00000,50.0,490.00000
6,0.0,0.00000,0,0.000,0.00000,0.0,0.00000
7,30.0,30.00000,1,0.000,330.00000,50.0,380.00000


## Test Scenario 14: Overstocked Start

Excess initial inventory - tests if solver correctly decides not to order.

In [16]:
# Scenario 14: Overstocked start
scenario_14 = {
    "V_max": 200.0,
    "P": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
    "C": 50.0,
    "D": [8.0, 9.0, 10.0, 8.0, 9.0, 10.0, 8.0],  # Low demand
    "I0": 150.0,  # Massive initial inventory
    "alpha": 0.05,  # Low loss
    "T": 7
}

inp14 = SolverInput(**scenario_14)

try:
    sol14 = solve(inp14)
    df14 = solution_to_dataframe(sol14, inp14)
    print(f"Objective value: {sol14.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol14.y) if y == 1]}")
    display(HTML(df14.to_html(index=False)))
    plot_scenario("Scenario 14: Overstocked Start", sol14, inp14, df14)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol14 = None
    df14 = None

Objective value: 0.00 PLN
Orders placed on days: []


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,8.0,0.0,0,134.500000,0.0,0,0.0
2,9.0,0.0,0,118.775000,0.0,0,0.0
3,10.0,0.0,0,102.836250,0.0,0,0.0
4,8.0,0.0,0,89.694437,0.0,0,0.0
5,9.0,0.0,0,76.209716,0.0,0,0.0
6,10.0,0.0,0,62.399230,0.0,0,0.0
7,8.0,0.0,0,51.279268,0.0,0,0.0


## Test Scenario 15: Exact Capacity Matching

Boundary test - orders designed to exactly hit warehouse capacity limit.

In [17]:
# Scenario 15: Exact capacity matching
scenario_15 = {
    "V_max": 100.0,  # Exact capacity
    "P": [5.0, 20.0, 20.0, 20.0, 20.0, 20.0, 5.0],  # Very low prices on days 1 and 7
    "C": 30.0,  # Low transport to encourage bulk buying
    "D": [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
    "I0": 0.0,  # Start empty
    "alpha": 0.05,  # Very low loss
    "T": 7
}

inp15 = SolverInput(**scenario_15)

try:
    sol15 = solve(inp15)
    df15 = solution_to_dataframe(sol15, inp15)
    print(f"Objective value: {sol15.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol15.y) if y == 1]}")
    display(HTML(df15.to_html(index=False)))
    plot_scenario("Scenario 15: Exact Capacity Matching", sol15, inp15, df15)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol15 = None
    df15 = None

Objective value: 440.37 PLN
Orders placed on days: [1]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,10.0,82.074828,1,72.074828,410.374142,30.0,440.374142
2,10.0,0.000000,0,58.471087,0.000000,0.0,0.000000
3,10.0,0.000000,0,45.547533,0.000000,0.0,0.000000
4,10.0,0.000000,0,33.270156,0.000000,0.0,0.000000
5,10.0,0.000000,0,21.606648,0.000000,0.0,0.000000
6,10.0,0.000000,0,10.526316,0.000000,0.0,0.000000
7,10.0,0.000000,0,0.000000,0.000000,0.0,0.000000


## Test Scenario 16: Two-Week Sprint with Mid-Week Break

14-day horizon with high demand in weeks 1-2 and low demand mid-week break.

In [18]:
# Scenario 16: Two-week sprint with break
# Week 1: High demand (5 days), Mid-break: Low demand (3 days), Week 2: Return to high (6 days)
demand_2week_break = [20, 22, 21, 19, 20] + [5, 5, 6] + [19, 21, 20, 22, 21, 20]
prices_2week_break = [12, 11, 10, 11, 12] + [15, 15, 15] + [11, 10, 11, 12, 11, 10]  # Higher during break

scenario_16 = {
    "V_max": 150.0,
    "P": prices_2week_break,
    "C": 65.0,
    "D": demand_2week_break,
    "I0": 25.0,
    "alpha": 0.09,
    "T": 14
}

inp16 = SolverInput(**scenario_16)

try:
    sol16 = solve(inp16)
    df16 = solution_to_dataframe(sol16, inp16)
    print(f"Objective value: {sol16.objective_value:.2f} PLN")
    print(f"Orders placed on days: {[i+1 for i, y in enumerate(sol16.y) if y == 1]}")
    display(HTML(df16.to_html(index=False)))
    plot_scenario("Scenario 16: Two-Week Sprint with Break", sol16, inp16, df16)
except SolverFail as e:
    print(f"Solver failed: {e}")
    sol16 = None
    df16 = None

Objective value: 2812.40 PLN
Orders placed on days: [2, 3, 9, 10, 13]


Day,Demand (kg),Ordered (kg),Order Placed,Inventory (kg),Purchase Cost (PLN),Transport Cost (PLN),Total Daily Cost (PLN)
1,20,0.000000,0,2.750000,0.000000,0.0,0.000000
2,22,19.497500,1,0.000000,214.472500,65.0,279.472500
3,21,89.572049,1,68.572049,895.720486,65.0,960.720486
4,19,0.000000,0,43.400564,0.000000,0.0,0.000000
5,20,0.000000,0,19.494513,0.000000,0.0,0.000000
6,5,0.000000,0,12.740007,0.000000,0.0,0.000000
7,5,0.000000,0,6.593407,0.000000,0.0,0.000000
8,6,0.000000,0,0.000000,0.000000,0.0,0.000000
9,19,19.000000,1,0.000000,209.000000,65.0,274.000000
10,21,69.544862,1,48.544862,695.448617,65.0,760.448617
